In [ ]:
import io
from datetime import timedelta
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    PauliFeatureMap,
    RealAmplitudes,
    ZFeatureMap,
    ZZFeatureMap,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit_algorithms.optimizers import Optimizer
from qiskit_machine_learning.algorithms import QSVC, VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from ucimlrepo import fetch_ucirepo
from utils import RANDOM_SEED, display_confussion_matrix, get_dataset, ExperimentLogger

# Desarrolla un QSVM

En este notebook probamos diferentes configuraciones de QSVMx parapoder elegir la más óptima.

Al igual que siempre, fijamos la semilla de los números aleatorios para que los resultados sean reproducibles.

In [2]:
algorithm_globals.random_seed = RANDOM_SEED

## Preprocesado del dataset

Utilizamos una función `get_dataset` del módulo `utilis.py` que hemos definido. El funcionamiento de esta función es exactamente el mismo que el de la sección de preprocesado del notebook [practica final](practica_final.ipynb).

In [3]:
X_train, y_train, X_test, y_test = get_dataset()

## Definición de funciones auxiliares

A continuación probamos diferentes clasificadores, los cuales se diferencian en el `feature_map`.

In [ ]:
def train_QSVC(feature_map, seed: int = RANDOM_SEED) -> QSVC:
    sampler = Sampler(seed=seed)
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    return _fit(qsvc)


def _fit(svc: SVC) -> SVC:
    global X_train, Y_train, X_test, Y_test

    start = time()
    svc.fit(X_train, Y_train)
    total_time = time() - start

    if total_time < 10:
        print(f"Tiempo total de entrenamiento: {total_time * 1000:.2} ms")
    else:
        print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return svc

## Probar diferentes modelos

In [ ]:
qsvc = train_QSVC(z_feature_map(number_of_features))
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:11:47.936480
SVC en entrenamiento: 0.96484
SVC en test (Accuracy): 0.96491
SVC en test (Recall): 0.94444


In [ ]:
qsvc = train_QSVC(zz_feature_map(number_of_features))
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:53:35.252097
SVC en entrenamiento: 1.0
SVC en test (Accuracy): 0.83333
SVC en test (Recall): 0.61111


In [ ]:
qsvc = train_QSVC(PauliFeatureMap(number_of_features).decompose())
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:42:16.785167
SVC en entrenamiento: 1.0
SVC en test (Accuracy): 0.78947
SVC en test (Recall): 0.61111
